# Quant Research One-Pager (Plotly)

This notebook mirrors the logic of `python-sdk/showcase/00_investor_onepager.py`, but produces research-friendly artifacts: tables, charts, and distributions.

Artifacts:
- Watchlist scan table (GO/NO-GO)
- Equity curve comparison: no-cost vs with-cost
- Trade return distribution
- Regime loss attribution


In [1]:
import os
import time
from datetime import datetime, timezone

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

from aipricepatterns import BacktestAuditor, Client

pd.set_option('display.max_columns', 50)
pd.set_option('display.width', 140)

def to_ms_utc(dt: str) -> int:
    # dt example: '2025-10-10 15:00' (interpreted as UTC)
    return int(datetime.strptime(dt, '%Y-%m-%d %H:%M').replace(tzinfo=timezone.utc).timestamp() * 1000)

def period_start_ms(days: int) -> int:
    return int((time.time() - days * 24 * 60 * 60) * 1000)


## Parameters
Tune these first.


In [2]:
BASE_URL = os.getenv('AIPP_BASE_URL', 'https://aipricepatterns.com/api/rust')
API_KEY = os.getenv('AIPP_API_KEY')

WATCHLIST = os.getenv('AIPP_RESEARCH_WATCHLIST', 'BTCUSDT,ETHUSDT,SOLUSDT').split(',')
WATCHLIST = [s.strip() for s in WATCHLIST if s.strip()]

INTERVAL = os.getenv('AIPP_RESEARCH_INTERVAL', '1h')
Q = int(os.getenv('AIPP_RESEARCH_Q', '60'))
F = int(os.getenv('AIPP_RESEARCH_F', '24'))
LIMIT = int(os.getenv('AIPP_RESEARCH_LIMIT', '16'))

DAYS = int(os.getenv('AIPP_RESEARCH_DAYS', '90'))
STEP = int(os.getenv('AIPP_RESEARCH_STEP', '24'))
MIN_PROB = float(os.getenv('AIPP_RESEARCH_MIN_PROB', '0.50'))
TOP_K = int(os.getenv('AIPP_RESEARCH_TOP_K', '5'))

FEE_PCT = float(os.getenv('AIPP_FEE_PCT', '0.04'))
SLIPPAGE_PCT = float(os.getenv('AIPP_SLIPPAGE_PCT', '0.02'))

BLOCKED_REGIMES = os.getenv('AIPP_BLOCK_REGIMES', 'BEARISH_MOMENTUM,STABLE_DOWNTREND').split(',')
BLOCKED_REGIMES = {s.strip() for s in BLOCKED_REGIMES if s.strip()}

print('Base URL:', BASE_URL)
print('Watchlist:', WATCHLIST)
print(f'Interval: {INTERVAL}  q={Q}  f={F}  limit={LIMIT}')
print(f'Backtest: last {DAYS} days  step={STEP}  minProb={MIN_PROB}')
print(f'Friction: feePct={FEE_PCT}%  slippagePct={SLIPPAGE_PCT}%')
print('Blocked regimes:', sorted(BLOCKED_REGIMES))


Base URL: https://aipricepatterns.com/api/rust
Watchlist: ['BTCUSDT', 'ETHUSDT', 'SOLUSDT']
Interval: 1h  q=60  f=24  limit=16
Backtest: last 90 days  step=24  minProb=0.5
Friction: feePct=0.04%  slippagePct=0.02%
Blocked regimes: ['BEARISH_MOMENTUM', 'STABLE_DOWNTREND']


In [3]:
client = Client(base_url=BASE_URL, api_key=API_KEY)
auditor = BacktestAuditor(client)


## 1) Watchlist scan (batch search + current regime)
We pick a single candidate (best GO by match count).


In [4]:
reqs = [
    {'symbol': s, 'interval': INTERVAL, 'q': Q, 'f': F, 'limit': LIMIT, 'sort': 'similarity'}
    for s in WATCHLIST
]
batch = client.batch_search(reqs)
results = batch.get('results') if isinstance(batch, dict) else None
if not isinstance(results, list) or len(results) != len(WATCHLIST):
    results = [{} for _ in WATCHLIST]

rows = []
for i, sym in enumerate(WATCHLIST):
    search_res = results[i] if i < len(results) and isinstance(results[i], dict) else {}
    matches = search_res.get('matches') or []
    match_count = len(matches) if isinstance(matches, list) else 0

    regime_id = None
    regime_error = None
    try:
        r = client.get_current_regime(sym, INTERVAL)
        regime_id = ((r or {}).get('currentRegime') or {}).get('id') if isinstance(r, dict) else None
    except Exception as e:
        regime_error = str(e)

    decision = 'NO'
    reason = ''
    if regime_error:
        reason = f'regime_error:{regime_error}'
    elif regime_id and regime_id in BLOCKED_REGIMES:
        reason = f'blocked_regime:{regime_id}'
    elif match_count == 0:
        reason = 'no_matches'
    else:
        decision = 'GO'
        reason = 'ok'

    rows.append({
        'symbol': sym,
        'regime': regime_id,
        'matches': match_count,
        'decision': decision,
        'reason': reason,
    })

scan_df = pd.DataFrame(rows).sort_values(['decision', 'matches'], ascending=[True, False])
scan_df


,symbol,regime,matches,decision,reason
0,BTCUSDT,BULLISH_MOMENTUM,16,GO,ok
1,ETHUSDT,BEARISH_MOMENTUM,16,NO,blocked_regime:BEARISH_MOMENTUM
2,SOLUSDT,BEARISH_MOMENTUM,16,NO,blocked_regime:BEARISH_MOMENTUM


In [5]:
go_candidates = scan_df[scan_df['decision'] == 'GO'].copy()
PICK = go_candidates.sort_values('matches', ascending=False).head(1)['symbol'].iloc[0] if len(go_candidates) else None
print('Selected candidate:', PICK)

Selected candidate: BTCUSDT


## 2) Grid intel (optional)


In [6]:
grid = None
if PICK:
    try:
        grid = client.get_grid_stats(PICK, INTERVAL)
    except Exception as e:
        print('Grid stats error:', e)
grid


{'confidence': {'components': {'corrScore': 0.9709,
   'directionalScore': 0.0,
   'rmseScore': 0.9361},
  'label': 'medium',
  'score': 0.6224},
 'distribution': {'expectedMax': 2.302,
  'expectedMin': -1.443,
  'max': 13.7623,
  'mean': 0.5573,
  'median': 0.8714,
  'min': -12.897,
  'samples': 50,
  'stdev': 3.7146},
 'gridLevels': [{'label': '-3σ', 'pct': -10.5865},
  {'label': '-2σ', 'pct': -6.8719},
  {'label': '-1σ', 'pct': -3.1573},
  {'label': 'mean', 'pct': 0.5573},
  {'label': '+1σ', 'pct': 4.272},
  {'label': '+2σ', 'pct': 7.9866},
  {'label': '+3σ', 'pct': 11.7012}],
 'gridRecommendation': {'bias': 'balanced',
  'centerPct': 0.5573,
  'comment': 'Up-move probability 50.00%',
  'levels': 60,
  'lowerPct': -6.8719,
  'suggestedStepPct': 0.2476,
  'upperPct': 7.9866},
 'meta': {'algo': 'corr_v1',
  'baseOffset': 0,
  'cursor': 0,
  'filters': None,
  'forecastHorizon': 30,
  'interval': '1h',
  'lastPrice': 88256.08,
  'limit': 10,
  'nextCursor': 10,
  'queryLength': 40,
  '

## 3) Backtest reality check (no-cost vs with-cost)
We run two walk-forward backtests on the same period, then compare equity curves and trade distributions.


In [7]:
if not PICK:
    raise RuntimeError('No GO candidate in watchlist. Lower blocklist / min filters or change the watchlist.')

start_ts = period_start_ms(DAYS)

bt_no_cost = client.backtest(
    symbol=PICK,
    interval=INTERVAL,
    q=Q,
    f=F,
    step=STEP,
    top_k=TOP_K,
    min_prob=MIN_PROB,
    start_ts=start_ts,
    include_stats=True,
    fee_pct=0.0,
    slippage_pct=0.0,
)

bt_with_cost = client.backtest(
    symbol=PICK,
    interval=INTERVAL,
    q=Q,
    f=F,
    step=STEP,
    top_k=TOP_K,
    min_prob=MIN_PROB,
    start_ts=start_ts,
    include_stats=True,
    fee_pct=FEE_PCT,
    slippage_pct=SLIPPAGE_PCT,
)

s0 = bt_no_cost.get('stats') if isinstance(bt_no_cost, dict) else {}
s1 = bt_with_cost.get('stats') if isinstance(bt_with_cost, dict) else {}

summary = pd.DataFrame([
    {'scenario': 'no_cost', 'totalReturnPct': s0.get('totalReturnPct'), 'winRate': s0.get('winRate'), 'profitFactor': s0.get('profitFactor'), 'maxDrawdownPct': s0.get('maxDrawdownPct')},
    {'scenario': 'with_cost', 'totalReturnPct': s1.get('totalReturnPct'), 'winRate': s1.get('winRate'), 'profitFactor': s1.get('profitFactor'), 'maxDrawdownPct': s1.get('maxDrawdownPct')},
])
summary


,scenario,totalReturnPct,winRate,profitFactor,maxDrawdownPct
0,no_cost,-5.955950,43.820225,0.958029,15.780615
1,with_cost,-25.858547,47.191011,0.715408,29.277668


In [8]:
curve0 = client.equity_curve_to_df(bt_no_cost)
curve1 = client.equity_curve_to_df(bt_with_cost)

fig = go.Figure()
if not curve0.empty:
    y0 = curve0['equity'] if 'equity' in curve0.columns else curve0.iloc[:, 0]
    fig.add_trace(go.Scatter(x=curve0.index, y=y0, mode='lines', name='No-cost'))
if not curve1.empty:
    y1 = curve1['equity'] if 'equity' in curve1.columns else curve1.iloc[:, 0]
    fig.add_trace(go.Scatter(x=curve1.index, y=y1, mode='lines', name='With-cost'))
fig.update_layout(title=f'Equity Curve — {PICK} ({INTERVAL})', xaxis_title='Time', yaxis_title='Equity', height=420)
fig


### Trade return distribution


In [9]:
trades = (bt_with_cost.get('results') or bt_with_cost.get('trades') or []) if isinstance(bt_with_cost, dict) else []
trades = trades if isinstance(trades, list) else []
trade_returns = [t.get('actualReturnPct') for t in trades if isinstance(t, dict) and isinstance(t.get('actualReturnPct'), (int, float))]
trade_df = pd.DataFrame({'actualReturnPct': trade_returns})
trade_df.describe(percentiles=[0.05, 0.25, 0.5, 0.75, 0.95])


,actualReturnPct
count,89.000000
mean,-0.306067
std,2.439052
min,-8.085069
5%,-4.161868
25%,-1.690525
50%,-0.290139
75%,1.006965
95%,3.089718
max,7.177505


In [10]:
if len(trade_df):
    fig = px.histogram(trade_df, x='actualReturnPct', nbins=40, title='Trade Return Distribution (with costs)')
    fig.update_layout(height=380)
    fig
else:
    print('No trades to plot.')


## 4) Regime audit (loss attribution)
We attribute losing trades to market regimes.


In [11]:
trades_for_audit = []
for t in trades:
    if not isinstance(t, dict):
        continue
    ts = t.get('ts')
    if not isinstance(ts, int):
        continue
    trades_for_audit.append({'entryTime': ts, 'returnPct': t.get('actualReturnPct', 0.0)})

report = auditor.analyze_losses(PICK, INTERVAL, trades_for_audit)
report


Analyzing 47 losing trades...


{'total_losses': 47,
 'regime_distribution': {'STABLE_DOWNTREND': 13,
  'MEAN_REVERSION': 5,
  'BULLISH_MOMENTUM': 10,
  'STABLE_TRENDING': 7,
  'BEARISH_MOMENTUM': 12}}

In [12]:
dist = (report or {}).get('regime_distribution') or {}
total_losses = int((report or {}).get('total_losses') or 0)

if isinstance(dist, dict) and dist:
    dist_df = pd.DataFrame([{'regime': k, 'lossCount': int(v)} for k, v in dist.items()])
    dist_df = dist_df.sort_values('lossCount', ascending=False)
    fig = px.bar(dist_df.head(12), x='regime', y='lossCount', title=f'Regime Loss Attribution (N={total_losses})')
    fig.update_layout(height=380)
    fig
else:
    print('No regime distribution available.')


## Notes for researchers
- If you want to replay a specific historical moment, compute a timestamp via `to_ms_utc('YYYY-MM-DD HH:MM')` and use `aipp rl-episodes --anchor-ts ...` to retrieve context-aware historical episodes around that anchor.
- For production usage, prefer confidence gating: only act when similarity is high enough and the episode ensemble has acceptable downside metrics.
